In [2]:
import requests
import urllib.parse
from bs4 import BeautifulSoup

In [4]:
url="https://www.jobbank.gc.ca/jobsearch/jobsearch?flg=E&fcan=1&fss=1&fss=C&fwcl=B&fwcl=C&fwcl=D&fwcl=E&fglo=1&sort=M&fwht=D&fwht=M&fsrc=16&fjnc=1&fexp=0&fexp=1&fprov=AB&fprov=ON&fskl=101020&fskl=101010"
page=requests.get(url)

soup=BeautifulSoup(page.content, "html.parser")

job_elements=soup.find_all("a", class_="resultJobItem")

print(job_elements)

[<a class="resultJobItem" href="/jobsearch/jobposting/41447597;jsessionid=92FDB773BFA786BD59639BDAF5C1FE71.jobsearch77?source=searchresults" id="ajaxupdateform:j_id_34_3_3p:0:j_id_34_3_3r">
<h3 class="title">
<span class="flag">
<span class="new">
						New
					</span>
</span>
<span class="job-source job-source-icon-16"><span class="wb-inv">Job Bank</span></span>
<span class="noctitle"> farm worker, general
				
					<span class="job-marker">
<span class="verified">
<span aria-hidden="true" class="fa-stack">
<span class="fas fa-circle fa-stack-2x"></span>
<span class="fas fa-check fa-stack-1x fa-inverse fa-icon"></span>
</span>
<span class="wb-inv">Verified</span>
</span>
<span class="description"><span aria-hidden="true" class="fa fa-info-circle"></span>This job was posted directly by the employer on Job Bank.</span>
</span>
</span>
</h3>
<ul class="list-unstyled">
<li class="date">July 12, 2024
			</li>
<li class="business">Ferme Philos inc.</li>
<li class="location"><span aria-hidd

In [11]:
role_list=[]
dates_list=[]
business_list=[]
locations_list=[]
salaries_list=[]
id_list=[]
noccode_list=[]

jobs={
    "ROLES":role_list,
    "PUBLISHED DATES":dates_list,
    "BUSINESS":business_list,
    "LOCATIONS":locations_list,
    "SALARY":salaries_list,
    "ID":id_list,
    "NOC CODE":noccode_list
}

for job_list in job_elements:
    
    role=job_list.find("span", class_="noctitle")
    fixed_role=role.find(string=True, recursive=False).strip() #Cuando usas Recursive, no pone los datos de los demás "span".
    role_list.append(fixed_role)
    
    published_date=job_list.find("li", class_="date")
    fixed_date=published_date.find(string=True, recursive=False).strip() 
    dates_list.append(fixed_date)

    business_name=job_list.find(class_="business")
    fixed_business=business_name.find(string=True, recursive=False).strip()
    business_list.append(fixed_business)

    location=job_list.find("li", class_="location").get_text()
    fixed_location=location[10:].strip()
    locations_list.append(fixed_location)

    salary=job_list.find(class_="salary").get_text()
    fixed_salary=salary[30:].strip()
    if "hourly" in fixed_salary:
        fix_sal=fixed_salary[:5]
    if "anually" in fixed_salary:
        fix_sal=(fixed_salary[:9].replace(",",""))/2000
    salaries_list.append(f"$"+fix_sal)

for link in soup.find_all('a'): 
    id=link.get('href')
    try:
        id.split("/")[-1:]
        id_number=id[22:30]
        try:
            id_number=int(id_number)
            id_list.append(id_number)
        except:
            pass
    except:
        pass
        
        

print(str(dates_list)+"\n")
print(str(role_list)+"\n")
print(str(business_list)+"\n")
print(str(locations_list)+"\n")
print(str(salaries_list)+"\n")
print(id_list)

['July 12, 2024', 'July 10, 2024', 'July 10, 2024', 'July 09, 2024', 'July 08, 2024', 'July 08, 2024', 'July 08, 2024', 'July 08, 2024', 'July 07, 2024', 'July 05, 2024', 'July 04, 2024', 'July 04, 2024', 'July 04, 2024', 'July 04, 2024', 'July 04, 2024', 'July 03, 2024', 'July 02, 2024', 'July 02, 2024', 'July 01, 2024', 'June 30, 2024', 'June 30, 2024', 'June 29, 2024', 'June 27, 2024', 'June 27, 2024', 'June 27, 2024']

['farm worker, general', 'food service supervisor', 'cook', 'beef cattle farm worker', 'kitchen helper', 'supervisor, food services', 'shift manager - fast food restaurant', 'farm supervisor', 'truck dispatcher', 'delivery drivers supervisor', 'fruit farm labourer', 'long haul truck driver', 'long haul truck driver', 'cook', 'car mechanic', 'nursery worker', 'kitchen helper', 'poultry farm worker', 'kitchen helper', 'general construction labourer', 'drywall installer', 'food counter attendant', 'long haul truck driver', 'farm worker - harvesting', 'personal support w

In [17]:
for ids in id_list:
    id_url="https://www.jobbank.gc.ca/jobsearch/jobposting/"+str(ids)
    print(id_url)
    try:
        id_page=requests.get(id_url)
        id_soup=BeautifulSoup(id_page.content, "html.parser")
        print(id_soup)
        noccode=id_soup.find("span", class_="aa_jobbank_")
        fixed_noccode=role.find(string=True, recursive=False).strip()
        noccode_list.append(fixed_noccode)
    except:
        pass
    break
    

print(noccode_list)

https://www.jobbank.gc.ca/jobsearch/jobposting/41447597
<!DOCTYPE html>

<html class="no-js" dir="ltr" lang="en"><head id="j_id_5">
<meta charset="utf-8"/>
<title>farm worker, general - Sarsfield, ON - Job posting - Job Bank
			      
		</title>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="eng" name="dcterms.language"/>
<meta content="Employment and Social Development Canada" name="dcterms.creator"/>
<meta content="ESDC-EDSC_JobBank-GuichetEmplois" name="dcterms.service"/>
<meta content="2" name="dcterms.accessRights"/><meta content="" name="dcterms.title"/>
<meta content="" property="og:title"/>
<meta content="Look at this job opportunity for a farm worker, general at Ferme Philos inc. At Sarsfield, null or search through thousands of other job postings on Job Bank, Canada’s one-stop job board." name="description" property="og:description"/>
<meta content="/jobsearch/jobposting/41447597" property="og:url"/>
<meta content="https://www.jobbank.gc.c

In [150]:
print(jobs)

{'ROLES': ['farm worker, general', 'food service supervisor', 'cook', 'beef cattle farm worker', 'kitchen helper', 'supervisor, food services', 'shift manager - fast food restaurant', 'truck dispatcher', 'delivery drivers supervisor', 'long haul truck driver', 'long haul truck driver', 'cook', 'car mechanic', 'nursery worker', 'kitchen helper', 'poultry farm worker', 'food counter attendant', 'farm worker - harvesting', 'personal support worker - home support', 'long haul truck driver', 'home support worker', 'assistant cook', 'personal care provider - home care', 'light duty cleaner', 'long haul truck driver'], 'PUBLISHED DATES': ['July 12, 2024', 'July 10, 2024', 'July 10, 2024', 'July 09, 2024', 'July 08, 2024', 'July 08, 2024', 'July 08, 2024', 'July 07, 2024', 'July 05, 2024', 'July 04, 2024', 'July 04, 2024', 'July 04, 2024', 'July 04, 2024', 'July 03, 2024', 'July 02, 2024', 'July 02, 2024', 'June 29, 2024', 'June 27, 2024', 'June 27, 2024', 'June 26, 2024', 'June 25, 2024', 'Ju